In [ ]:
from ditto_server import DittoStrategy
from ditto_client import ditto_client_fn_generator
from femnist import download_femnist
from utils import CLIENT_MODEL_DIR

import shutil
import os

import flwr as fl
import numpy as np

In [ ]:
# Download FEMNIST data (does nothing if already present)
download_femnist()

In [ ]:
results = {}

for _lambda in np.arange(0, 1.1, 0.1):
    # Reset client models
    if os.path.exists(CLIENT_MODEL_DIR):
        shutil.rmtree(CLIENT_MODEL_DIR)  # Delete client model directory
    os.makedirs(CLIENT_MODEL_DIR)  # Recreate client model directory
    
    num_clients = 8
    num_rounds = 25
    epochs_per_round = 25
    
    # https://flower.ai/docs/framework/tutorial-series-customize-the-client-pytorch.html
    results[_lambda] = fl.simulation.start_simulation(
        num_clients=num_clients,
        client_fn=ditto_client_fn_generator(_lambda=_lambda, epochs_per_round=epochs_per_round),
        config=fl.server.ServerConfig(num_rounds=num_rounds),
        strategy=DittoStrategy(print_progress=False),
        client_resources={
            # 'num_cpus': max(os.cpu_count()//num_clients, 1),
            'num_gpus': 1/num_clients
        }
    )

In [ ]:
results